In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import torch
torch.cuda.is_available()

True

# Process

In [3]:
with open("/data/shengkuan/CS172/cs172-assignment2-2023fall-skkk256/splits/eigen_zhou/val_files.txt", "r+") as f:
    lines = f.readlines()
new_lines = []
for line in lines:
    if "09_26" in line:
        new_lines.append(line)
len(new_lines)

1387

In [4]:
len(new_lines)
with open("/data/shengkuan/CS172/cs172-assignment2-2023fall-skkk256/splits/eigen_26/val_files.txt", "w+") as f:
    f.write("".join(new_lines))

# Test

In [3]:
import os
import argparse
import torch

In [4]:
from trainer import Trainer
from options import MonodepthOptions

In [5]:
args = MonodepthOptions()
args = args.parser.parse_args([])

In [6]:
args.data_path = "/data/shengkuan/CS172/cs172-assignment2-2023fall-skkk256/data/kitti_data"
args.log_dir = "/data/shengkuan/CS172/cs172-assignment2-2023fall-skkk256/log"
args.png = True
args.num_workers = 0
# args.batch_size = 1

In [ ]:
trainer = Trainer(args)

In [8]:
train_data = trainer.train_loader

In [ ]:
train_data_iter = iter(train_data)
next(train_data_iter)

In [ ]:
trainer.train()

# Test Feature Extractor

In [2]:
%matplotlib inline

import os
import numpy as np
import PIL.Image as pil
import matplotlib.pyplot as plt

import torch
from torchvision import transforms

import networks
from modules.utils import download_model_if_doesnt_exist

In [35]:
from networks.feature_encoder import FeatureEncoder
from networks.feature_decoder import FeatureDecoder
def build_extractor(num_layers, pretrained_path):
    encoder = FeatureEncoder(num_layers, None)
    decoder = FeatureDecoder(encoder.num_ch_enc)
    if pretrained_path is not None:
        checkpoint = torch.load(pretrained_path, map_location='cpu')
        for name, param in encoder.state_dict().items():
            encoder.state_dict()[name].copy_(checkpoint['state_dict']['Encoder.' + name])
        for param in encoder.parameters():
            param.requires_grad = False
        
        for name, param in decoder.state_dict().items():
            decoder.state_dict()[name].copy_(checkpoint['state_dict']['Decoder.' + name])
        for param in decoder.parameters():
            param.requires_grad = False
    return encoder, decoder

    
# encoder = FeatureEncoder(50)
encoder, decoder = build_extractor(50, "/data/shengkuan/CS172/cs172-assignment2-2023fall-skkk256/models/autoencoder.pth");

In [43]:
# image_path = "/data/shengkuan/CS172/cs172-assignment2-2023fall-skkk256/data/kitti_data/2011_09_26/2011_09_26_drive_0002_sync/image_02/data/0000000069.png"
image_path = "assets/test_image.jpg"

input_image = pil.open(image_path).convert('RGB')
original_width, original_height = input_image.size

# feed_height = encoder['height']
# feed_width = encoder['width']
# input_image_resized = input_image.resize((feed_width, feed_height), pil.LANCZOS)

input_image_pytorch = transforms.ToTensor()(input_image).unsqueeze(0)
input_image_pytorch.shape

torch.Size([1, 3, 235, 638])

In [44]:
feature = encoder(input_image_pytorch)

In [45]:
outputs = decoder(feature, 0)

In [46]:
outputs[('res_img', 0, 0)][0].shape

torch.Size([3, 256, 640])

In [47]:
feature_np = np.transpose(outputs[('res_img', 0, 0)][0].detach().numpy(), (1, 2, 0))

In [48]:
diff_np = feature_np

In [ ]:
from PIL import Image

feature_image = Image.fromarray((feature_np * 255).astype(np.uint8))
feature_image;

In [16]:
feature[4].shape

torch.Size([4, 2048, 6, 20])

In [31]:
pretrain_model = torch.load("models/autoencoder.pth")

In [32]:
state_keys = pretrain_model["state_dict"].keys()